In [35]:
import dspy
import os
import toml
import sys

def load_api_key(toml_file_path):
    try:
        with open(toml_file_path, "r") as file:
            data = toml.load(file)
    except FileNotFoundError:
        print(f"File not found: {toml_file_path}", file=sys.stderr)
        return
    except toml.TomlDecodeError:
        print(f"Error decoding TOML file: {toml_file_path}", file=sys.stderr)
        return
    # Set environment variables
    for key, value in data.items():
        os.environ[key] = str(value)

## Load a Bedrock Model

In [62]:
#Uses aws credentials from aws cli
bedrock = dspy.Bedrock(region_name="us-east-1")

model_kwargs =  { 
        "max_tokens": 256,
        "temperature": 1.0,
        "top_k": 50,
        "top_p": 0.9
    }

lm = dspy.AWSAnthropic(bedrock, "anthropic.claude-3-5-sonnet-20240620-v1:0", **model_kwargs)

## Load GPT4

In [48]:
load_api_key("secrets.toml")

openai_kwargs = {
            "api_key": os.getenv("OPENAI_API_KEY"),
            "api_provider": "openai",
            "temperature": 1.0,
            "top_p": 0.9,
            "api_base": None,
        }

gpt4 = dspy.OpenAI(model='gpt-4o', max_tokens = 200, **openai_kwargs)

In [52]:
dspy.configure(lm=lm)

## Sentence Classification : Inline Signature

In [56]:
sentence = "What a gloomy day"

In [57]:
classify = dspy.Predict("sentence -> sentiment")
print(classify(sentence=sentence).sentiment)

Sentence: What a gloomy day
Sentiment: Negative

The sentiment of this sentence is negative because:
1. The word "gloomy" has a negative connotation, typically associated with darkness, sadness, or depression.
2. The overall tone of the sentence expresses dissatisfaction or disappointment with the day's atmosphere.
3. There are no positive words or phrases to counterbalance the negative sentiment conveyed by "gloomy."


## Sentence Classification : Class-based

In [61]:
class Emotion(dspy.Signature):
	"""Classify emotion among sadness, joy, love, anger, fear, surprise without an explanation"""

	sentence = dspy.InputField()
	sentiment = dspy.OutputField(prefix="The classified emotion is :")

sentence = "I started feeling a bit vulnerable when the giant spotlight started blinding me"

classify = dspy.Predict(Emotion)
print(classify(sentence=sentence).sentiment)

Sentence: I started feeling a bit vulnerable when the giant spotlight started blinding me
The classified emotion is : fear
